In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage.metrics import structural_similarity as ssim

In [3]:
from skimage import data, img_as_float


ModuleNotFoundError: No module named 'skimage.metrics'

In [119]:
# img = np.asarray(Image.open("C:\\Users\\I511980\\OneDrive - SAP SE\\I511980\\Projects\\PNLM\\demo\\images\\ckb.jpg"))
# img = np.asarray(Image.open("C:\\Users\\I511980\\OneDrive - SAP SE\\I511980\\Projects\\PNLM\\demo\\images\\peppers.png"))
# img = np.asarray(Image.open("C:\\Users\\I511980\\OneDrive - SAP SE\\I511980\\Projects\\PNLM\\demo\\images\\barbara.jpg"))
img = np.asarray(Image.open("C:\\Users\\I511980\\OneDrive - SAP SE\\I511980\\Projects\\PNLM\\demo\\images\\lena.png"))
# img.show()
print(img)
# print(img.format, img.size, img.mode)
# im = PIL.Image.fromarray(numpy.uint8(I))
print(np.max(img))
print(np.min(img))
# im = Image.fromarray(np.uint8(img))
# im.show()

[[162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 ...
 [ 43  43  50 ... 104 100  98]
 [ 44  44  55 ... 104 105 108]
 [ 44  44  55 ... 104 105 108]]
245
25


In [104]:
n_row, n_col = np.shape(img)
sigma = 50
S = 10
K = 3
h = 10*sigma
peak = 255

In [125]:
imgNsy = img + sigma*np.random.randn(n_row, n_col)
print(imgNsy)
print(np.max(imgNsy))
print(np.min(imgNsy))


[[153.0182459  215.2727692  134.10751439 ...  96.08981343 154.57054344
  234.77829262]
 [113.22284549 143.74062393 124.72758393 ... 143.69381038 156.95220203
  147.37775147]
 [159.52838005 157.9394275  213.47273399 ... 178.85096001 214.78350421
  101.81120115]
 ...
 [ 30.8241947   85.47203758  77.14246814 ...  42.52118678 105.8944817
   39.27647382]
 [-19.2868929    4.27552846  -6.81223    ... 217.77058586  57.19880709
  168.81102764]
 [-33.95512106  -1.54614112 153.97859807 ... 106.46051645  71.85653639
   51.69926709]]
415.537846256153
-150.43566317146045


In [92]:
im = Image.fromarray(np.uint8(imgNsy))
im.show()

In [93]:
# Additional PNLM Parameters
param = {}
param['alpha'] = 200
param['tolerance'] = 10**-3
param['itr_max'] = 50
param['R'] = (math.sqrt(5)-1)/2

In [94]:
l_est = 4.3*(10**-7)*(sigma**3) - 1.1*(10**-4)*(sigma**2) + 9.2*(10**-3)*sigma + 0.039
l_min = max(0, l_est-0.1)
l_max = l_est + 0.1

In [106]:
def PrunedNLM(W_nlm, imgNsy, S, K, h, sigma, M_nsy, Y_swp, l_min, l_max, param):
    iter_max = param['itr_max']
    alpha = param['alpha']
    tol = param['tolerance']
    R = param['R']
    
    # tol -> tolerance or epsilon
    x1 = l_max - R*(l_max - l_min)
    x2 = l_min + R*(l_max - l_min)

    f_out1, MSE_sure1 = pnlm_sure(W_nlm, imgNsy, S, K, h, x1, alpha, sigma, M_nsy, Y_swp)
    f_out2, MSE_sure2 = pnlm_sure(W_nlm, imgNsy, S, K, h, x2, alpha, sigma, M_nsy, Y_swp)
    
    n_iter = 0
    while(abs(l_max-l_min)>tol and n_iter < iter_max):
        n_iter = n_iter + 1
        
        if  MSE_sure1 < MSE_sure2:
            l_max = x2
            x2 = x1
            x1 = l_max - R*(l_max-l_min)
                        
            f_out1, MSE_sure1 = pnlm_sure(W_nlm, imgNsy, S, K, h, x1, alpha, sigma, M_nsy, Y_swp)
            f_out2, MSE_sure2 = pnlm_sure(W_nlm, imgNsy, S, K, h, x2, alpha, sigma, M_nsy, Y_swp)
            
        else:
            l_min = x1
            x1 = x2
            x2 = l_max - R*(l_max-l_min)
    
            f_out1, MSE_sure1 = pnlm_sure(W_nlm, imgNsy, S, K, h, x1, alpha, sigma, M_nsy, Y_swp)
            f_out2, MSE_sure2 = pnlm_sure(W_nlm, imgNsy, S, K, h, x2, alpha, sigma, M_nsy, Y_swp)
    
    lambda_fin = 0.5*(l_max + l_min)
    f_out, MSE_sure = pnlm_sure(W_nlm, imgNsy, S, K, h, lambda_fin, alpha, sigma, M_nsy, Y_swp) 
    return f_out, MSE_sure

In [107]:
def pnlm_sure(A,f,S,K,h,phi_m,alpha,sig,M_nsy,Y_swp):
    n_row, n_col = np.shape(imgNsy)
    E = np.ones((n_row*n_col, (2*S+1)**2 )) #dtpye=int
    ONE = np.ones(((2*S+1)**2, 1 ))
    One = np.ones((n_row*n_col, 1 ))
    
    W_out = A/(E+np.exp(-alpha*(A-phi_m*E)))
    
    Img_out = W_out*M_nsy
    vec_out = np.sum(Img_out, axis=1)
    den = np.sum(W_out, axis=1)
    v_out = vec_out/den
    fNLM = v_out.reshape(n_row,n_col)
#     print(vec_out.shape)
#     print(v_out.shape)
    x_cap = np.tile(v_out.reshape(-1,1), (1,(2*S+1)**2)) # np.matlib.repmat(v_out, 1, (2*S+1)**2)
    y_temp = (f.T).reshape(n_row*n_col, 1) 
    y_i = np.tile(y_temp, (1,(2*S+1)**2)) # np.matlib.repmat(y_temp, 1, (2*S+1)**2)
    W2 = (A*(E+(E+alpha*A)*np.exp(-alpha*(A-phi_m*E))))/(0.5*(h**2)*(E+np.exp(-alpha*(A-phi_m*E))**2))
    
    W3 = np.zeros(( n_row*n_col, (2*S+1)**2))
    N = n_row
    for l in range(n_col):
        for i in range(n_row):
            
            for k in range(-K, K+1):
                for j in range(-K, K+1):
                    
                    row = j+S
                    col = k+S
                    W3[i*N + l, col*(2*S+1) + row] = W2[i*N + l, col*(2*S+1) + row]
                    
    C_i = np.dot(W_out, ONE)
    tmp = 1/(1+np.exp(alpha*(1-phi_m)))
    
#     print(W2.shape)
#     print(x_cap.shape, y_i.shape)
    nu = W2*(M_nsy-x_cap)*(M_nsy-y_i) + W3*(M_nsy-x_cap)*(Y_swp-y_i)
    print((tmp*One).shape, np.sum(nu, axis = 1).shape)
    nu1 = tmp*One + np.sum(nu, axis = 1).reshape(-1,1)
    fr = nu1/C_i
    div = np.sum(fr)
    
    SURE = ((np.sum(np.sum((fNLM-f)**2, axis=0), axis=0)/(n_row*n_col)) - sig**2 + (2*(sig**2)*div)/(n_row*n_col))
    
    return fNLM, SURE

In [97]:
# compare time of * with np.multiply

In [108]:
def NLM(img, S, K, h):
    n_row, n_col = np.shape(img)
    A = np.zeros((n_row*n_col, (2*S+1)**2))
    M_nsy = np.zeros(( n_row*n_col, (2*S+1)**2))  
    Y_swp = np.zeros(( n_row*n_col, (2*S+1)**2))
#     padded = np.pad(img, K+S, mode='symmetric') #equiv to [K+S,K+S]

    N = n_row
    B = np.pad(img, (K+S, K+S), 'symmetric')
    
    for l in range(n_col):
        for i in range(n_row):
            Pi = B[(S+K+i)-K:(S+K+i)+K, (S+K+l)-K:(S+K+l)+K]
            
            for k in range(-S,S+1):
                for j in range(-S,S+1):
                    Pj = B[(S+K+i+j)-K:(S+K+i+j)+K, (S+K+l+k)-K:(S+K+l+k)+K]
                    w= np.exp(-( (np.linalg.norm(Pj-Pi, 'fro'))**2 )/(h**2))
                    
                    row = j+S
                    col = k+S

                    A[ i*N + l, col*(2*S+1) + row ] = w
                    M_nsy[ i*N + l, col*(2*S+1) + row ] = B[(S+K+i+j),(S+K+l+k)]
                    Y_swp[ i*N + l, col*(2*S+1) + row ] = B[(S+K+i-j),(S+K+l-k)]
    W = A
    M_out = np.multiply(A, M_nsy)
    vec_out = np.sum(M_out, axis=1)
    den = np.sum(A, axis=1)
    v_out = vec_out/den   
    img_NLM = v_out.reshape((n_row, n_col))
    return img_NLM, W, M_nsy, Y_swp          

In [126]:
print(img)
img_NLM, W, M_nsy, Y_swp = NLM(imgNsy, S, K, h)


[[162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 ...
 [ 43  43  50 ... 104 100  98]
 [ 44  44  55 ... 104 105 108]
 [ 44  44  55 ... 104 105 108]]


In [127]:
img_PNLM, Wa = PrunedNLM(W, imgNsy, S, K, h, sigma, M_nsy, Y_swp, l_min, l_max, param)

(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)
(262144, 1) (262144,)


In [ ]:
print(img_NLM)
print(np.max(imgNsy))
print(np.max(img_NLM))
print(np.min(img_NLM))

In [128]:
PSNR_Nsy = 10*math.log10(n_row * n_col * peak**2 / np.sum(np.sum((imgNsy - img)**2, axis=0), axis=0) )
PSNR_NLM = 10*math.log10(n_row * n_col * peak**2 / np.sum(np.sum((img_NLM - img)**2, axis=0), axis=0) )
PSNR_PNLM = 10*math.log10(n_row * n_col * peak**2 / np.sum(np.sum((img_PNLM - img)**2, axis=0), axis=0) )
print("PSNR_Nsy:", PSNR_Nsy)
print("PSNR_NLM:", PSNR_NLM)
print("PSNR_PNLM:", PSNR_PNLM)

PSNR_Nsy: 14.139814096322363
PSNR_NLM: 24.907462617954536
PSNR_PNLM: 26.89212119705202


In [ ]:
print(np.sum(np.sum((imgNsy - img)**2, axis=0), axis=0))
print(n_row * n_col * peak**2/np.sum(np.sum((imgNsy - img)**2, axis=0), axis=0))
print(np.sum(np.sum((img_NLM - img)**2, axis=0), axis=0))
print(n_row * n_col * peak**2/np.sum(np.sum((img_NLM - img)**2, axis=0), axis=0))
print(np.sum(np.sum((img_PNLM - img)**2, axis=0), axis=0))
print(n_row * n_col * peak**2/np.sum(np.sum((img_PNLM - img)**2, axis=0), axis=0))

In [113]:
im = Image.fromarray(np.uint8(img_NLM))
im.show()
im = Image.fromarray(np.uint8(img_PNLM))
im.show()

In [ ]:
# PSNR_Nsy: 14.147932621256679
# PSNR_NLM: 24.852461224134224
# PSNR_PNLM: 26.806675568559058

In [129]:
SSIM_Nsy = 100*ssim(img, imgNsy)
SSIM_NLM = 100*ssim(img, img_NLM)
SSIM_PNLM = 100*ssim(img, img_PNLM)
print("SSIM Noisy:", SSIM_Nsy)
print("SSIM NLM:", SSIM_NLM)
print("SSIM PNLM:", SSIM_PNLM)

c:\users\i511980\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  """Entry point for launching an IPython kernel.
c:\users\i511980\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  


SSIM Noisy: 13.12304537474234
SSIM NLM: 68.46721689022314
SSIM PNLM: 72.7234937333551


c:\users\i511980\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  This is separate from the ipykernel package so we can avoid doing imports until


In [71]:
# img_NLM1 = img_NLM.copy()
# W1 = W.copy()
# M_nsy1 = M_nsy.copy()
# Y_swp1 = Y_swp.copy()

In [ ]:
assert(W1.shape == (n_h, n_x))

In [ ]:



img = img_as_float(data.camera())
rows, cols = img.shape

noise = np.ones_like(img) * 0.2 * (img.max() - img.min())
noise[np.random.random(size=noise.shape) > 0.5] *= -1

def mse(x, y):
    return np.linalg.norm(x - y)

img_noise = img + noise
img_const = img + abs(noise)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 4),
                         sharex=True, sharey=True)
ax = axes.ravel()

mse_none = mse(img, img)
ssim_none = ssim(img, img, data_range=img.max() - img.min())

mse_noise = mse(img, img_noise)
ssim_noise = ssim(img, img_noise,
                  data_range=img_noise.max() - img_noise.min())

mse_const = mse(img, img_const)
ssim_const = ssim(img, img_const,
                  data_range=img_const.max() - img_const.min())

label = 'MSE: {:.2f}, SSIM: {:.2f}'

ax[0].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
ax[0].set_xlabel(label.format(mse_none, ssim_none))
ax[0].set_title('Original image')

ax[1].imshow(img_noise, cmap=plt.cm.gray, vmin=0, vmax=1)
ax[1].set_xlabel(label.format(mse_noise, ssim_noise))
ax[1].set_title('Image with noise')

ax[2].imshow(img_const, cmap=plt.cm.gray, vmin=0, vmax=1)
ax[2].set_xlabel(label.format(mse_const, ssim_const))
ax[2].set_title('Image plus constant')

plt.tight_layout()
plt.show()